In [11]:
import pandas as pd
import requests
import time

In [12]:
api_key = 'RGAPI-0b437417-9c5d-462b-8402-c3e3cc8329c2'
summoner_name = '학장주유소'

In [13]:
requesturl = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key
requests.get(requesturl).json()

{'id': 'FWJXIFLSt6WtPPuZ_y9UeLHTdnRqj_jwo4XQFkWQkIQth1U',
 'accountId': 'D6u62CpEWag_XxSzT-bEQF5-T7d2mtZUAyxXMT6W9_e9Ais',
 'puuid': '2CIbiJ46u2meb-qaLoVFC8QTN5vU26PWRRkLBN5n_SFnBRaB_SarMMAqARqRRxTE_Ju9CthevWrzoA',
 'name': '학장주유소',
 'profileIconId': 5766,
 'revisionDate': 1692380221000,
 'summonerLevel': 251}

In [14]:
summoner_puuid = requests.get(requesturl).json()['puuid']

In [15]:
games_list = []
index = 0
while True:
    requesturl = 'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+'/ids?queue=420&start='+str(index)+'&count=10&api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

In [16]:
games_list

['KR_6597443448',
 'KR_6597407402',
 'KR_6597371976',
 'KR_6585775901',
 'KR_6585729803',
 'KR_6576920126',
 'KR_6576857423',
 'KR_6576814681',
 'KR_6569720853',
 'KR_6569656755']

In [25]:
# 아이템 구매 순서 정보 추출
item_purchase_orders = []

for match_id in games_list:
    match_url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response = requests.get(match_url)
    match_data = response.json()
    

    for participant in match_data['info']['participants']:
        if participant['summonerName'] == '학장주유소':
            items_purchased = participant['itemsPurchased']
            item_slots = [participant[f'item{i}'] for i in range(7)]  # item0 ~ item6
            print(f"Items Purchased: {items_purchased}")
            print(f"Item Slots: {item_slots}")

Items Purchased: 15
Item Slots: [3095, 3006, 1036, 1042, 1038, 0, 3340]
Items Purchased: 12
Item Slots: [2065, 3859, 2055, 1001, 0, 0, 3364]
Items Purchased: 17
Item Slots: [1103, 0, 3078, 3047, 1029, 0, 3364]
Items Purchased: 32
Item Slots: [6676, 6691, 3047, 3071, 3095, 1038, 3364]
Items Purchased: 29
Item Slots: [6333, 3156, 3047, 3078, 0, 6672, 3364]
Items Purchased: 26
Item Slots: [2055, 3190, 3857, 3023, 3075, 3047, 3364]
Items Purchased: 23
Item Slots: [2055, 2065, 3860, 3011, 3047, 1029, 3364]
Items Purchased: 15
Item Slots: [0, 2065, 3860, 1029, 3047, 0, 3364]
Items Purchased: 36
Item Slots: [3142, 3814, 3042, 6693, 3111, 6676, 3364]
Items Purchased: 21
Item Slots: [6671, 3009, 3095, 3094, 1018, 3134, 3363]


In [27]:
def get_item_name(item_id):
    versions_url = "https://ddragon.leagueoflegends.com/api/versions.json"
    last_version = requests.get(versions_url).json()[0]

    items_url = f"http://ddragon.leagueoflegends.com/cdn/{last_version}/data/ko_KR/item.json"
    items_data = requests.get(items_url).json()["data"]

    item_name = items_data.get(str(item_id), {}).get("name", "Unknown Item")
    return item_name

# Usage
for i in item_slots:  # Replace with the actual item ID
    item_name = get_item_name(i)
    print(f"Item Name: {item_name}")

Item Name: 돌풍
Item Name: 신속의 장화
Item Name: 폭풍갈퀴
Item Name: 고속 연사포
Item Name: 민첩성의 망토
Item Name: 톱날 단검
Item Name: 망원형 개조
